# Set up

In [52]:
!git clone https://github.com/NLP-Reichman/assignment_1.git
!mv assignment_1/data data
!rm assignment_1/ -r

'git' is not recognized as an internal or external command,
operable program or batch file.
'mv' is not recognized as an internal or external command,
operable program or batch file.
'rm' is not recognized as an internal or external command,
operable program or batch file.


# Introduction
In this assignment you will be creating tools for learning and testing language models. The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.
The relevant files are under the data folder:

- en.csv (or the equivalent JSON file)
- es.csv (or the equivalent JSON file)
- fr.csv (or the equivalent JSON file)
- in.csv (or the equivalent JSON file)
- it.csv (or the equivalent JSON file)
- nl.csv (or the equivalent JSON file)
- pt.csv (or the equivalent JSON file)
- tl.csv (or the equivalent JSON file)

In [53]:
import json
from google.colab import files
import pandas as pd

# Implementation

## Part 1
Implement the function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. Our token definition is a single UTF-8 encoded character. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data.

Note - do NOT lowercase the sentences in whi HW.

In [54]:
import os
start_token = '\uE000'  # Using Private Use Area character
end_token = '\uE001'    # Using Private Use Area character
def preprocess() -> list[str]:
  '''
  Return a list of characters, representing the shared vocabulary of all languages
  '''
  data_folder_path = "./data"

  unique_characters = set()
  # Iterate through all files in the data folder
  for filename in os.listdir(data_folder_path):

      # Check if the file is a JSON file
      if filename.endswith(".json"):
          # print(filename)
          # Get the full path to the file
          file_path = os.path.join(data_folder_path, filename)

          # Open the file and read its contents
          with open(file_path, "r", encoding='utf-8-sig') as f:
              # Load the JSON data as a uTF-8 encoded string
              json_data = json.load(f)
              for key, val in json_data["tweet_text"].items():
                #   print(key, val)
                  for c in val:
                    unique_characters.add(c) # Update the hash with the token in utf-8 encoding

                      


  # add the 2 special chars to the set:               
  unique_characters.add(start_token)
  unique_characters.add(end_token)
  return sorted(list(unique_characters))




In [55]:
vocabulary = preprocess()
vocabulary_length = len(vocabulary)
# print(f"Vocabulary length: {vocabulary_length}")

## Part 2
Implement the function *lm* that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant *n*-1 sequences, and the values are dictionaries with the *n*_th tokens and their corresponding probabilities to occur. For example, for a trigram model (tokens are characters), it should look something like:

{ "ab":{"c":0.5, "b":0.25, "d":0.25}, "ca":{"a":0.2, "b":0.7, "d":0.1} }

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

In [56]:
class inner_gram_dict(dict):
    '''
    A subclassed dictionary to store the probabilities of the nth charecter in an n-gram
    To save memory, we will store the size of the vocabulary for the key
     which is linked with this dict - and as a result will be able to return 1/V
     for missing values without having to store the entire vocabulary explicitly.
     This saves a lot of memory when the analyzed corpus is large.
    '''
    def __init__(self, vocabulary_size=0, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.vocabulary_size = vocabulary_size

    def set_total_count(self, total_count):
        self.total_count = total_count

    def __missing__(self, key):
    #    self[key] = self.default_probability
        return 1 / self.vocabulary_size
    
def lm(lang: str, n: int, smoothed=False) -> dict[str, dict[str, float]]:
    '''
    Return a language model for the given lang and n_gram (n)
    :param lang: the language of the model
    :param n: the n_gram value
    :return: a dictionary where the keys are n_grams and the values are dictionaries
              with the n_th tokens and their corresponding probabilities to occur
    '''
    model = {}
    model['Unknown'] = inner_gram_dict(vocabulary_length if (smoothed == True)  else 0)
    data_folder_path = "./data"
    filename = f"{lang}.json"
    file_path = os.path.join(data_folder_path, filename)
    
    # Open the file and read its contents
    with open(file_path, "r", encoding='utf-8-sig') as f:
        json_data = json.loads(f.read())
        # Iterate through each sentence in the corpus
        for key, sentence in json_data["tweet_text"].items():
            # Iterate through each n-gram in the sentence
            # add n-1 start tokens and 1 end token to the sentence:
            sentence = start_token * (n-1) + sentence + end_token
          
            for i in range(len(sentence) - n + 1):
                ngram = sentence[i:i+n-1]
                next_token = sentence[i+n-1]

                # Check if the n-gram is not already in the model
                if ngram not in model:
                    model[ngram] = {}
                # handle next_token:
                if next_token not in model[ngram]:
                    model[ngram][next_token] = 0
                model[ngram][next_token] += 1
                    
    
    # Calculate the probabilities for each n-gram
    for ngram in model:
        if smoothed:
            # add all unseen chars from the vocabulary to the model, with a value of 1 (smoothing):
            for char in vocabulary:
                if char in model[ngram]:
                    model[ngram][char] += 1      

        total_count = sum(model[ngram].values())
        if smoothed:
            total_count += vocabulary_length
        for token in model[ngram]:
            model[ngram][token] /= total_count
    
    return model


In [57]:
# res = lm("en",2)
# print(f'unsmoothed len res: {len(res)}')

# res = lm("en",3, True)
# print(f'smoothed len res: {len(res)}')

# res = lm("fr",3, smoothed=True)
# print(f'smoothed len res: {len(res)}')
# # print(res['a'])

# res = lm("es",3)
# print(f'unsmoothed len res: {len(res)}')



## Part 3
Implement the function *eval* that returns the perplexity of a model (dictionary) running over the data file of the given target language.

In [58]:
import math
import numpy as np
def eval(model: dict, target_lang: str, n: int) -> float:
  '''
  Return the perplexity value calculated over applying the model on the text file
  of the target_lang language.
  :param model: the language model
  :param target_lang: the target language
  :return: the perplexity value
  '''
  # n = len(list(model.keys())[0]) + 1

  data_folder_path = "./data"
  filename = f"{target_lang}.json"
  file_path = os.path.join(data_folder_path, filename)
  perplexity = 0
  # Open the file and read its contents
  with open(file_path, "r", encoding='utf-8-sig') as f:
      json_data = json.loads(f.read())
      # Iterate through each sentence in the corpus
      perplexities = []
      for key, sentence in json_data["tweet_text"].items():
        sentence = start_token * (n-1) + sentence + end_token
        sentence_enthropy = 0
        # probabilities= [] (only to test the calculation)
        for i in range(len(sentence) - n + 1):
            ngram = sentence[i:i+n-1]
            next_token = sentence[i+n-1]
            if ngram in model:
                if next_token in model[ngram]:
                    # probabilities.append(model[ngram][next_token])
                    sentence_enthropy +=  ( -1 * np.log2(model[ngram][next_token]))
                else:
                    # probabilities.append(1/vocabulary_length)
                    sentence_enthropy += -1 * np.log2(1/vocabulary_length) # # todo  review
                    # throw an exception if the token is not in the model's keys
                    # raise Exception(f"{next_token} NOT IN MODEL for {ngram}")
                    # sentence_enthropy +=  ( -1 * math.log2(model[ngram]['unknown']))
            else:
                #raise an exception if the ngram is not in the model's keys
                # raise Exception("NOT IN MODEL")
                # probabilities.append(1/vocabulary_length)
                sentence_enthropy += -1 * np.log2(1/vocabulary_length) # # todo  review
              
        # entropies = [-np.log2(prop) for prop in probabilities]
        # entropy = np.average(entropies)
        # perplexity = 2 ** entropy
        # # assert similarity between perplexity and the perplexity calculated from the sentence's entropy
        # # do not use == but use some tolerance, as the calculation of the perplexity is not accurate
        # assert abs(perplexity - 2 ** (sentence_enthropy/(len(sentence) - n + 1))) < 0.0001
        
        perplexities.append(2 ** (sentence_enthropy/(len(sentence) - n + 1)))
        # print(f'{sentence}\nperplexity: {perplexities[-1]}')
        
  res =  np.average(perplexities)
  
#   print(f'perplexity: {res}')
  return res
  


In [59]:
# for n in range(3, 4):
#     model = lm("en", n, smoothed=True)
#     for eval_lang in ["en", "fr", "es"]:

#         perplexity =  eval(model, eval_lang, n)
#         print(f"Eval lang {eval_lang} on en  SMOOTHED n=={n} : Perplexity: {perplexity}")

## Part 4
Implement the *match* function that calls *eval* using a specific value of *n* for every possible language pair among the languages we have data for. You should call *eval* for every language pair four times, with each call assign a different value for *n* (1-4). Each language pair is composed of the source language and the target language. Before you make the call, you need to call the *lm* function to create the language model for the source language. Then you can call *eval* with the language model and the target language. The function should return a pandas DataFrame with the following four columns: *source_lang*, *target_lang*, *n*, *perplexity*. The values for the first two columns are the two-letter language codes. The value for *n* is the *n* you use for generating the specific perplexity values which you should store in the forth column.

In [60]:
def match() -> pd.DataFrame:
  '''
  Return a DataFrame containing one line per every language pair and n_gram.
  Each line will contain the perplexity calculated when applying the language model
  of the source language on the text of the target language.
  :return: a DataFrame containing the perplexity values
  '''
  perplexity_values = []
  for n in range(1, 5):
    for model_lang in ["en", "es", "fr", "in", "it", "nl", "pt", "tl"]:
      model = lm(model_lang, n, True)

      for lang in ["en", "es", "fr", "in", "it", "nl", "pt", "tl"]:
        perplexity = eval(model, lang, n)
        perplexity_values.append({"source": model_lang, "target": lang, "n": n, "perplexity": perplexity})
        print(f'source: {model_lang}, target: {lang}, n: {n}, perplexity: {perplexity}')
  return pd.DataFrame(perplexity_values)

# df = match()

In [61]:
# print(df[3]['en'])

## Part 5
Implement the *generate* function which takes a language code, *n*, the prompt (the starting text), the number of tokens to generate, and *r*, which is the random seed for any randomized action you plan to take in your implementation. The function should start generating tokens, one by one, using the language model of the given source language and *n*. The prompt should be used as a starting point for aligning on the probabilities to be used for generating the next token.

Note - The generation of the next token should be from the LM's distribution.

In [62]:
import random
models_1 = {}
# for each language create a model with n=1, no smoothing:
for lang in ["en", "es", "fr", "in", "it", "nl", "pt", "tl"]:
  models_1[lang] = lm(lang, 1, False)

def get_next_token(model, ngram, lang):
  '''
  Return the next token based on the given model, ngram, and random value from 0 to 1.
  '''
  # iterate over the model[ngram] and sum the probabilities until the sum is greater than rand:
  sum = 0
  # get a random number between 0 to 1:
  rand = random.random()
  try:
    for token, prob in model[ngram].items():
      sum += prob
      if sum > rand:
        return token
  except:
    # reutrn the most common token in language if ngram is not in the model:
    return get_next_token(models_1[lang], '', lang)


def generate(lang: str, n: int, prompt: str, number_of_tokens: int, r: int) -> str:
  '''
  Generate text in the given language using the given parameters.
  :param lang: the language of the model
  :param n: the n_gram value
  :param prompt: the prompt to start the generation
  :param number_of_tokens: the number of tokens to generate
  :param r: the random seed to use
  '''
  # initialize random seed with r
  random.seed(r)
  model = lm(lang, n)
  generated_text = prompt
  # handle short prompts (pad with start tokens)
  if len(generated_text) < n - 1:
    generated_text = start_token * (n - 1 - len(generated_text)) + generated_text
   
  for i in range(number_of_tokens):
    ngram = generated_text[-n+1:] # TODO handle the case of shorter prompts
    # get next token based on the model and the dustribution in model[ngram] . use the random seed
    next_token = get_next_token(model, ngram, lang)
    # max(model[ngram], key=model[ngram].get)
    generated_text += next_token
    # break if the end token is generated:
    if next_token == end_token:
      break
  return generated_text

# print(generate("en", 3, "I am ", 20, 5))
# print(generate("fr", 3, "je suis ", 20, 5))

## Part 6
Play with your generate function, try to generate different texts in different language and various values of *n*. No need to submit anything of that.

In [63]:
# print(generate("fr", 2, "je suis ", 20, 5))
# print(generate("fr", 2, "je suis ", 20, 5))
# print(generate("fr", 2, "je suis", 20, 5))

# print(generate("en", 2, "I am ", 20, 5))
# print(generate("en", 2, "I am", 20, 5))

# Testing

Copy the content of the **tests.py** file from the repo and paste below. This will create the results.json file and download it to your machine.

In [64]:
####################
# PLACE TESTS HERE #
# Create tests
def test_preprocess():
    return {
        'vocab_length': len(preprocess()),
    }

def test_lm():
    return {
        'english_2_gram_length': len(lm('en', 2, True)),
        'english_3_gram_length': len(lm('en', 3, True)),
        'french_3_gram_length': len(lm('fr', 3, True)),
        'spanish_3_gram_length': len(lm('es', 3, True)),
    }

def test_eval():
    return {
        'en_en': eval(lm('en', 3, True), 'en', 3),
        'en_fr': eval(lm('en', 3, True), 'fr', 3),
        'en_tl': eval(lm('en', 3, True), 'tl', 3),
        'en_nl': eval(lm('en', 3, True), 'nl', 3),
    }

def test_match():
    df = match()
    return {
        'en_en_3': df[(df['source'] == 'en') & (df['target'] == 'en') & (df['n'] == 3)]['perplexity'].values[0],
        'en_tl_3': df[(df['source'] == 'en') & (df['target'] == 'tl') & (df['n'] == 3)]['perplexity'].values[0],
        'en_nl_3': df[(df['source'] == 'en') & (df['target'] == 'nl') & (df['n'] == 3)]['perplexity'].values[0],
    }

def test_generate():
    return {
        'english_1_gram': generate('en', 1, "I", 20, 5), 
        'english_2_gram': generate('en', 2, "I am", 20, 5),
        'english_3_gram': generate('en', 3, "I am", 20, 5),
        'english_4_gram': generate('en', 4, "I Love", 20, 5),
        'spanish_2_gram': generate('es', 2, "Soy", 20, 5),
        'spanish_3_gram': generate('es', 3, "Soy", 20, 5),
        'french_2_gram': generate('fr', 2, "Je suis", 20, 5),
        'french_3_gram': generate('fr', 3, "Je suis", 20, 5),
    }

TESTS = [test_preprocess, test_lm, test_eval, test_match, test_generate]

# Run tests and save results
res = {}
for test in TESTS:
    try:
        cur_res = test()
        res.update({test.__name__: cur_res})
    except Exception as e:
        res.update({test.__name__: repr(e)})

with open('results.json', 'w') as f:
    json.dump(res, f, indent=2)

# Download the results.json file
files.download('results.json')

####################

source: en, target: en, n: 1, perplexity: 42.76682461040326
source: en, target: es, n: 1, perplexity: 42.67046214659845
source: en, target: fr, n: 1, perplexity: 43.14063470698611
source: en, target: in, n: 1, perplexity: 45.81344053143406
source: en, target: it, n: 1, perplexity: 44.045960028952855
source: en, target: nl, n: 1, perplexity: 47.228668850780814
source: en, target: pt, n: 1, perplexity: 44.59434819329026
source: en, target: tl, n: 1, perplexity: 52.659587034271645
source: es, target: en, n: 1, perplexity: 46.43514746773465
source: es, target: es, n: 1, perplexity: 41.125669499400104
source: es, target: fr, n: 1, perplexity: 42.442137184256
source: es, target: in, n: 1, perplexity: 48.40054336723995
source: es, target: it, n: 1, perplexity: 44.32664060784248
source: es, target: nl, n: 1, perplexity: 46.37009705225956
source: es, target: pt, n: 1, perplexity: 41.44749360308283
source: es, target: tl, n: 1, perplexity: 51.91310100711027
source: fr, target: en, n: 1, perplexi

NameError: name 'files' is not defined

In [ ]:
# Show the local files, results.json should be there now and
# also downloaded to your local machine
!ls -l

In [ ]:
# temp = lm("en", 4)
# print